In [10]:
import numpy as np
import tensorflow as tf

In [ ]:
class ConvLayer: # stride = 1
  def __init__(self, kernel_cnt, kernel_size):
    self.kernel_cnt = kernel_cnt
    self.kernel_size = kernel_size
    self.kernels = np.random.randn(kernel_cnt, kernel_size, kernel_size) / (kernel_size**2) # ref1a

  def patch_gen(self, image):
    h, w = image.shape
    self.image = image
    for h in range(h-self.kernel_size+1):
      for w in range(w-self.kernel_size+1):
        patch = image[h:h+self.kernel_size, w:w+self.kernel_size]
        yield patch, h, w # gen

  def forward(self, x):
    h, w = x.shape
    output = np.zeros((h-self.kernel_size+1, w-self.kernel_size+1, self.kernel_cnt))
    for patch, h, w in self.patch_gen(x):
      output[h,w] = np.sum(patch*self.kernels, axis=(1,2)) # ref1b
    return output
  
  def backward(self, dE_dy, lr):
    dE_dk = np.zeros(self.kernels.shape)
    for patch, h, w in self.patch_gen(self.image):
      for f in range(self.kernel_cnt):
        dE_dk[f] += patch * dE_dy[h, w, f]
    
    self.kernels -= lr*dE_dk

    return dE_dk

In [19]:
class MaxPoolLayer:
  def __init__(self, kernel_size):
    self.kernel_size = kernel_size

  def patch_gen(self, image):
    out_h = image.shape[0] // self.kernel_size
    out_w = image.shape[1] // self.kernel_size
    self.image = image

    for h in range(out_h):
      for w in range(out_w):
        patch = image[h*self.kernel_size:h*self.kernel_size+self.kernel_size, w*self.kernel_size:w*self.kernel_size+self.kernel_size]
        yield patch, h, w

  def forward(self, x):
    h, w, kernel_cnt = x.shape
    output = np.zeros((h//self.kernel_size, w//self.kernel_size, kernel_cnt))
    for patch, h, w in self.patch_gen(x):
      output[h,w] = np.amax(patch, axis=(0,1)) # ref2
    return output
  
  def backward(self, dE_dy):
    dE_dk = np.zeros(self.image.shape)
    for patch, h, w in self.patch_gen(self.image):
      h, w, kernel_cnt = patch.shape
      max = np.amax(patch, axis=(0,1))

      for ih in range(h):
        for iw in range(w):
          for ik in range(kernel_cnt):
            if patch[ih,iw,ik] == max[ik]:
              dE_dk[h*self.kernel_size+ih, w*self.kernel_size+iw, ik] = dE_dy[h, w, ik]

    return dE_dk

In [8]:
class SoftmaxLayer:
  def __init__(self, in_c, out_c):
    self.weight = np.random.randn(in_c, out_c)/in_c
    self.bias = np.zeros(out_c)

  def forward(self, x):
    self.ori_shape = x.shape
    flat = x.flatten()
    self.flat_in = flat
    output_1 = np.dot(flat, self.weight) + self.bias
    self.output = output_1
    softmax_output = np.exp(output_1) / np.sum(np.exp(output_1), axis=0)
    return softmax_output
  
  def backward(self, dE_dy, lr):
    for i, grad in enumerate(dE_dy):
      if grad == 0:
        continue
      transform = np.exp(self.output)
      S = np.sum(transform)

      dy_dz = -transform[i] * transform / (S**2)
      dy_dz[i] = transform[i] * (S - transform[i]) / (S**2)

      dz_dw = self.flat_in
      dz_db = 1
      dz_dx = self.weight

      dE_dz = grad * dy_dz

      dE_dw = dz_dw[np.newaxis].T @ dE_dz[np.newaxis]
      dE_db = dE_dz * dz_db
      dE_dX = dz_dx @ dE_dz

      self.weight -= lr*dE_dw
      self.bias -= lr*dE_db
 
    return dE_dX.reshape(self.ori_shape)

In [11]:
def forward(image, label, layers):
  output = image/255
  for layer in layers:
    output = layer.forward(output)
  
  loss = -np.log(output[label])
  acc = 1 if np.argmax(output) == label else 0
  
  return output, loss, acc

In [13]:
def backward(grad, layers, lr=5e-2):
  grad = grad
  for layer in layers[::-1]:
    if type(layer) in [ConvLayer, SoftmaxLayer]:
      grad = layer.backward(grad, lr)
    elif type(layer) == MaxPoolLayer:
      grad = layer.backward(grad)
    
    return grad

In [14]:
def train(image, label, layers, lr=5e-2):
  output, loss, acc = forward(image, label, layers)

  grad = np.zeros(10)
  grad[label] = -1/output[label]

  grad_back = backward(grad, layers, lr)

  return loss, acc

In [27]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train[:5000]
y_train = y_train[:5000]

# Define the network
layers = [
            ConvLayer(16,3), # layer with 8 3x3 filters, output (26,26,16)
            MaxPoolLayer(2), # pooling layer 2x2, output (13,13,16)
            SoftmaxLayer(13*13*16, 10) # softmax layer with 13*13*16 input and 10 output
        ] 

for epoch in range(100):
  print('Epoch {} ->'.format(epoch+1))
  # Shuffle training data
  permutation = np.random.permutation(len(X_train))
  X_train = X_train[permutation]
  y_train = y_train[permutation]
  # Training the CNN
  loss = 0
  accuracy = 0
  for i, (image, label) in enumerate(zip(X_train, y_train)):
    if i % 1000 == 0:
      print("Step {} average loss {}, accuracy {}%".format(i+1, loss/1000, accuracy/10))
      loss = 0
      accuracy = 0
    loss_1, accuracy_1 = train(image, label, layers)
    loss += loss_1
    accuracy += accuracy_1

Epoch 1 ->
Step 1 average loss 0.0, accuracy 0.0%
Step 1001 average loss 2.014081067518112, accuracy 30.7%
Step 2001 average loss 1.782994766683324, accuracy 39.9%
Step 3001 average loss 1.7267509236455436, accuracy 43.9%
Step 4001 average loss 1.647880029700954, accuracy 46.0%
Epoch 2 ->
Step 1 average loss 0.0, accuracy 0.0%
Step 1001 average loss 1.5748087444846675, accuracy 48.6%
Step 2001 average loss 1.5793160853375938, accuracy 46.8%
Step 3001 average loss 1.573720743040674, accuracy 47.3%
Step 4001 average loss 1.53357469226331, accuracy 49.5%
Epoch 3 ->
Step 1 average loss 0.0, accuracy 0.0%
Step 1001 average loss 1.5636079742992943, accuracy 49.0%
Step 2001 average loss 1.4726490035267488, accuracy 51.7%
Step 3001 average loss 1.5579761381663786, accuracy 48.5%
Step 4001 average loss 1.4939525352036018, accuracy 48.9%
Epoch 4 ->
Step 1 average loss 0.0, accuracy 0.0%
Step 1001 average loss 1.429630158699921, accuracy 52.4%
Step 2001 average loss 1.5576642091869246, accuracy 4